In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.random_graphs import newman_watts_strogatz_graph

In [9]:
data_df = pd.read_csv("data/ml-100k/u.data", sep="\t", header=None, names=["user_id", "item_id", "rating", "timestamp"])
data_df["user_id"] = data_df["user_id"].map({b: a for a, b in enumerate(data_df["user_id"].unique())})
data_df["item_id"] = data_df["item_id"].map({b: a for a, b in enumerate(data_df["item_id"].unique())})
data_df.head()

R = np.zeros((data_df["user_id"].nunique(), data_df["item_id"].nunique()))
for _, row_df in data_df.iterrows():
    user_id = row_df["user_id"]
    item_id = row_df["item_id"]
    rating = row_df["rating"]
    R[user_id, item_id] = rating
    
R = (R > 0).astype(int)
O = R.dot(R.T)
O = (O > 0).astype(int)
np.fill_diagonal(O, 0)

G = nx.Graph()
V = data_df["user_id"].unique()
G.add_nodes_from(V)

E = list(zip(*np.where(O)))
G.add_edges_from(E)

In [67]:
n = 1000 # nr. of nodes
k = 5 # nr. of nearest neighbors
p = 0.01 # rewiring probability
G = newman_watts_strogatz_graph(n, k, p)

In [175]:
N = nx.number_of_nodes(G)
degree_counts = Counter(dict(nx.degree(G)).values())
p = np.zeros(N)
for deg, count in degree_counts.items():
    p[deg] = count / N

In [69]:
def G_prime_0(x):
    n = len(p)
    expectation = 0.0
    for k in range(1, n):
        expectation += k * p[k] * np.power(x, k - 1)
    return expectation

In [70]:
def G_primeprime_0(x):
    n = len(p)
    expectation = 0.0
    for k in range(2, n):
        expectation += k * (k - 1) * p[k] * np.power(x, k - 2)
    return expectation

In [71]:
def G_prime_1(x, alpha=1):
    n = len(p)
    return alpha * (1.0 / G_prime_0(1)) * G_primeprime_0(x)

In [72]:
def G_1(x, alpha=1):
    return alpha * G_prime_0(x) / G_prime_0(1)

In [870]:
z1 = G_prime_0(1)
print("z1: %f" % z1)

z1: 4.046000


In [871]:
z2 = G_primeprime_0(1)
print("z2: %f" % z2)

z2: 12.368000


In [872]:
l = (np.log((N - 1) * (z2 - z1) + np.power(z1, 2)) - np.log(np.power(z1, 2))) / np.log(z2 / z1)
print("l: %f" % l)

l: 5.577463


In [873]:
alpha = 1
alpha / G_prime_0(1) * G_prime_0(alpha) * G_primeprime_0(1)

12.368

In [874]:
G_prime_0(1)

4.045999999999999

In [971]:
def G_prime_m(x, m, alpha=1, depth=0):
    if m <= 1:
        return m * G_prime_0(1)
    elif m < 2 and m > 1:
        return (alpha * z2 / z1) ** (m-1) * z1
    elif m > 1:
        return G_prime_m(G_1(x, alpha), m-1, alpha, depth+1) * G_prime_1(1, alpha)

m = 3
z_m = G_prime_m(1, m, alpha=0.5)
z_m

9.45176866040534

In [985]:
alphas = [0.01, 0.05, 0.1, 0.2]
for a in alphas:
    M = np.arange(1, 5, 0.05)
    L = [G_prime_m(1, m, alpha=a) for m in M]
    plt.scatter(M, L, s=1, label=r"$\alpha=%.2f$" % a)
plt.ylabel(r"$z_m$")
plt.xlabel(r"$m \in \mathbb{R}$")
plt.legend()
plt.show()

In [364]:
%matplotlib qt
alphas = [0.5, 0.7, 0.8, 0.9, 0.95, 0.99]
ms = np.arange(1, 10, 1)
for alpha in alphas:
    z_ms_alpha = []
    for m in ms:
        z_m = G_prime_m(1, m, alpha)
        z_ms_alpha.append(z_m)
    
    plt.plot(ms, 1 + np.cumsum(z_ms_alpha), label=r"$\alpha$=%.2f" % alpha)
plt.legend()
plt.ylabel(r"$1 + \Sigma z_m$")
plt.xlabel("$m$")
plt.show()

In [316]:
summation = 0.0
for m in range(1, 6):
    z_m = G_prime_m(1, m, alpha=1)
    summation += z_m
summation

523.0724814939947

In [365]:
from scipy import integrate
f = lambda m: G_prime_m(1., m, alpha=1)
integrate.quad(f, a=1., b=5.)

TypeError: must be real number, not NoneType